In [228]:
import pandas as pd
import numpy as np

elas = \
    pd.read_csv("C:\\Users\\amondal\\model_exp\\prev_month_family_values.tsv", sep="\t")

In [229]:
curr_fv = \
    pd.read_csv("C:\\Users\\amondal\\model_exp\\family_values.tsv", sep="\t")

In [230]:
cap_map = \
    pd.read_csv("C:\\Users\\amondal\\model_exp\\cap_map.tsv", sep="\t")

In [231]:
capacity = \
    pd.read_csv("C:\\Users\\amondal\\model_exp\\capacity.tsv", sep="\t")

In [232]:
discount_grps = \
    pd.read_csv("C:\\Users\\amondal\\model_exp\\resources\\discount_groups.tsv", sep="\t")

In [233]:
import json

with open("C:\\Users\\amondal\\model_exp\\resources\\geo_map_prod.json") as f:
    json_str = f.read()
    geo_map = json.loads(json_str)

In [234]:
with open("C:\\Users\\amondal\\model_exp\\resources\\sku_threshold_map.json") as f:
    json_str = f.read()
    sku_threshold = json.loads(json_str)

In [235]:
with open("C:\\Users\\amondal\\model_exp\\resources\\sku_type_map.json") as f:
    json_str = f.read()
    sku_type = json.loads(json_str)

In [236]:
with open("C:\\Users\\amondal\\model_exp\\resources\\price_constraints.json") as f:
    json_str = f.read()
    price_constraints = json.loads(json_str)

In [237]:
grps = discount_grps['discount_group'].to_list()
grps = [eval(x) for x in grps]
grp_id_map = {}
for i in range(len(grps)):
    for x in grps[i]:
        grp_id_map[x] = i

In [238]:
elas['dg'] = elas['pre_process_family'].apply(lambda x: grp_id_map[x])

In [239]:
elas['alpha'] = np.nan
elas['alpha'] = \
    (elas['OS'] == 'windows')*0.68 + \
    (elas['OS'] == 'linux')*1.06

In [240]:
elas['beta'] = np.nan
elas['beta'] = \
    (elas['OS'] == 'windows')*-1.97 + \
    (elas['OS'] == 'linux')*-2.19

In [241]:
elas['gamma'] = np.nan
elas['gamma'] = \
    (elas['OS'] == 'windows')*0.0 + \
    (elas['OS'] == 'linux')*0.0019

In [242]:
elas['factor'] = elas['spot_core_price'].pow(elas['alpha']*elas['weight'])

In [243]:
factors = \
    elas[['pre_process_family', 'OS', 'factor']]\
    .groupby(by=['pre_process_family', 'OS'],as_index=False)\
    .agg(prod_factors=('factor', 'prod'))

elas = pd.merge(elas, factors, on=['pre_process_family', 'OS'], how='left')

In [244]:
elas['demand_without_k'] = \
    elas['spot_core_price'].pow(elas['beta']) * elas['prod_factors'] / elas['factor']

In [245]:
elas['avg_daily_demand_w_nulls'] = \
    elas['totalDemand_ext_spot_nofraud'] / elas['totalDays_ext_spot_nofraud']

elas['avg_daily_demand_removed_nulls'] = \
    elas['avg_daily_demand_w_nulls'].fillna(0.0)

In [246]:
elas['od_avg_daily_demand_w_nulls'] = \
    elas['totalDemand_ext_od_nofraud'] / elas['totalDays_ext_od_nofraud']

elas['od_avg_daily_demand_removed_nulls'] = \
    elas['od_avg_daily_demand_w_nulls'].fillna(0.0)

In [247]:
elas['k'] = elas['avg_daily_demand_removed_nulls']/elas['demand_without_k']

In [248]:
elas[elas.k.isna()]

,Location,pre_process_family,OS,weight_check,spot_core_price,od_core_price,current_avg_discount,min_spot_core_price,max_spot_core_price,min_od_core_price,...,beta,gamma,factor,prod_factors,demand_without_k,avg_daily_demand_w_nulls,avg_daily_demand_removed_nulls,od_avg_daily_demand_w_nulls,od_avg_daily_demand_removed_nulls,k


In [249]:
# b = elas['avg_daily_demand_removed_nulls'].to_list()
# d = \
#     elas["k"]*elas["spot_core_price"].pow(elas['beta'])*elas["prod_factors"]/elas["factor"]
# d = d.to_list()
# e = elas["k"].to_list()
# f = elas["spot_core_price"].to_list()
# g = elas['beta'].to_list()
# h = elas["prod_factors"].to_list()
# j = elas["factor"].to_list()
# z = elas["spot_core_price"].pow(elas['alpha']*elas['weight']).to_list()

#print(list(zip(b[:1000], d[:1000], e[:1000], f[:1000], g[:1000], h[:1000], j[:1000], z[:1000])))

In [250]:
override_columns = list(set(curr_fv.columns) - {"Location", "pre_process_family", "OS"})

elas = \
    elas.rename(
    columns={
        col : f"{col}_pre_override"\
        for col in override_columns
    }
)

elas = pd.merge(elas, 
                curr_fv, 
                on=['Location', 'pre_process_family', 'OS'], 
                how="inner")

In [251]:
elas['od_core_price'].to_list()[:10]

[0.13335,
 0.1014,
 0.0678285714285714,
 0.1153749999999999,
 0.104625,
 0.1506249999999999,
 0.1353125,
 0.1813125,
 0.082494140625,
 0.128494140625]

In [252]:
elas['od_core_price'] = \
    ( 1.0 / ( 1.0 - elas['current_avg_discount'] ) ) * elas['spot_core_price']

elas['int_rounded_discount'] = \
    (100 * elas['current_avg_discount'].apply(lambda x : round(x, 2) )).apply(int)


In [253]:
elas['od_core_price'].to_list()[:10]

[0.13335442491790273,
 0.1014,
 0.06782857142857095,
 0.1153749999999999,
 0.10462500000000001,
 0.15062500360536585,
 0.13531250000000003,
 0.1813125007496699,
 0.08249414062499996,
 0.12849414062164993]

In [254]:
elas['int_rounded_discount'].to_list()[:10]

[49, 30, 90, 90, 90, 90, 90, 90, 74, 74]

In [255]:
k_vals =\
    elas[['Location', 'dg', 'OS', 'k']]\
    .groupby(['Location', 'dg', 'OS'], as_index=False)\
    .max()

k_vals['opt'] = k_vals['k'] < 0.00001

elas = \
    pd.merge(\
        elas,
        k_vals[['Location', 'dg', 'OS', 'opt']],
        how='inner',
        on = ['Location', 'dg', 'OS']
    )

In [256]:
elas_dg_er = \
    elas[['Location', 'dg', 'Evictions_FirstHourEvictedCores', 'Evictions_TotalCores']]\
    .groupby(by=['Location', 'dg'], as_index=False)\
    .agg(\
        Evictions_FirstHourEvictedCores_dg=('Evictions_FirstHourEvictedCores', 'sum'),
        Evictions_TotalCores_dg=('Evictions_TotalCores', 'sum'))

elas_dg_er['dg_er'] = \
    (elas_dg_er['Evictions_FirstHourEvictedCores_dg'])/(elas_dg_er['Evictions_TotalCores_dg']+1)

elas_dg_er['dg_high'] = elas_dg_er['dg_er'] >= 0.05

elas = pd.merge(elas, 
                elas_dg_er, 
                on=['Location', 'dg'], 
                how='inner')

In [257]:
elas[['dg', 'dg_er']].to_csv('test2.csv', sep=",")

In [258]:
elas['family_er'] = \
    (elas['Evictions_FirstHourEvictedCores'])/(elas['Evictions_TotalCores']+1)

elas['family_high'] = elas['family_er'] > 0.05

In [259]:
df5 = elas[elas.family_high != elas.dg_high][['Location', 'pre_process_family', 'OS', 'family_er', 'dg_er', 'family_high', 'dg_high']]
df5.to_csv("conflict_dg_family_er.csv", sep=",")

In [260]:
elas_dg_demand = \
    elas[['Location', 'dg', 'avg_daily_demand_removed_nulls']]\
    .groupby(by=['Location', 'dg'],as_index=False)\
    .agg(\
        dg_total_demand=('avg_daily_demand_removed_nulls', 'sum'))

elas = pd.merge(elas, 
                elas_dg_demand, 
                on=['Location', 'dg'], 
                how='inner')

In [261]:
elas['min_price_er'] = \
    (elas['dg_high'] * 0.9 * elas['spot_core_price']) + \
    (~elas['dg_high'] * 0.8 * elas['spot_core_price']) 

In [262]:
elas['max_price_er'] = \
    (elas['dg_high'] * 1.1 * elas['spot_core_price']) + \
    (~elas['dg_high'] * 1.05 * elas['spot_core_price']) 

In [263]:
elas['min_price_global'] = 0.1*elas['od_core_price']

In [264]:
elas['max_price_global'] = 0.7*elas['od_core_price']

In [265]:
elas[['min_price_er', 'max_price_er', 'min_price_global', 'max_price_global']]

,min_price_er,max_price_er,min_price_global,max_price_global
0,0.053924,0.070776,0.013335,0.093348
1,0.056784,0.074529,0.010140,0.070980
2,0.005426,0.007122,0.006783,0.047480
3,0.007377,0.009683,0.009182,0.064275
4,0.007377,0.009683,0.009182,0.064275
...,...,...,...,...
7986,0.097464,0.127921,0.060133,0.420931
7987,0.094158,0.123583,0.060065,0.420456
7988,0.099463,0.130546,0.064789,0.453520
7989,0.119989,0.157485,0.064761,0.453326


In [266]:
elas['min_price'] = elas[["min_price_er", "min_price_global"]].max(axis=1)

In [267]:
low_demand = elas['dg_total_demand'] <= 5000.0

elas['max_price'] = \
    low_demand*elas[["spot_core_price", "min_price"]].max(axis=1) + \
    ~low_demand*elas[["max_price_er", "max_price_global"]].min(axis=1)

small_diff = (elas['max_price'] - elas['min_price']).abs() <= 0.00001
elas['max_price'] = small_diff*elas['min_price'] + ~small_diff*elas['max_price']

In [268]:
elas[['Location', 'dg', 'OS', 'min_price', 'max_price']]

,Location,dg,OS,min_price,max_price
0,ae central,0,linux,0.053924,0.067405
1,ae central,0,windows,0.056784,0.070980
2,ae central,9,linux,0.006783,0.006783
3,ae central,9,linux,0.009182,0.009221
4,ae central,9,linux,0.009182,0.009221
...,...,...,...,...,...
7986,za west,25,windows,0.097464,0.121830
7987,za west,26,linux,0.094158,0.117698
7988,za west,26,linux,0.099463,0.124329
7989,za west,26,windows,0.119989,0.149986


In [269]:

elas['max_feasible_family_discount'] = \
    1.0 - ( elas['min_price'] / elas['od_core_price'] )

elas['min_feasible_family_discount'] = \
    1.0 - ( elas['max_price'] / elas['od_core_price'] )

elas['family_discount'] = elas['current_avg_discount']

dg_agg_1 =\
    elas\
        [['Location', 'dg', 'pre_process_family', 'avg_daily_demand_removed_nulls']]\
        .groupby(by=['Location', 'dg', 'pre_process_family'], as_index=False)\
        .sum()

dg_agg_2 =\
    dg_agg_1\
        [['Location', 'dg', 'avg_daily_demand_removed_nulls']]\
        .groupby(by=['Location', 'dg'], as_index=False)\
        .sum()\
        .rename(columns = {"avg_daily_demand_removed_nulls" : "total_daily_demand"})

elas = pd.merge(elas, dg_agg_2, how = 'inner', on = ['Location', 'dg'])

elas['family_prop_of_total_dg_demand'] = \
    elas['avg_daily_demand_removed_nulls'] / elas['total_daily_demand']

elas['family_prop_of_total_dg_demand'] = \
    elas['family_prop_of_total_dg_demand'].fillna(0.0)

In [270]:

current_discount_group_bounds =\
    elas[['Location', 'dg', 'OS', 'int_rounded_discount', 'min_feasible_family_discount', 'max_feasible_family_discount']]\
        .groupby(['Location', 'dg', 'OS', 'int_rounded_discount'], as_index=False)\
        .agg(
            {
                'max_feasible_family_discount' : 'min',
                'min_feasible_family_discount' : 'max',
            }
        )

elas =\
    elas\
    .rename(
        columns={
            'max_feasible_family_discount' : 'max_feasible_family_discount_pre_limit',
            'min_feasible_family_discount' : 'min_feasible_family_discount_pre_limit',
        }
    )\
    .merge(
        current_discount_group_bounds,
        how='inner',
        on = ['Location', 'dg', 'OS', 'int_rounded_discount']
    )

In [271]:
wm = lambda vec: np.average(vec, weights= (elas.loc[vec.index, "family_prop_of_total_dg_demand"] + 0.00001))

dg_elas = \
    elas\
    [['Location', 'OS', 'dg', 'max_feasible_family_discount', 'min_feasible_family_discount', 'family_discount']]\
    .groupby(['Location', 'OS', 'dg'], as_index=False)\
    .agg(
        {
            'max_feasible_family_discount' : 'min',
            'min_feasible_family_discount' : 'max',
            'family_discount' : wm
        }
    )\
    .rename(
        columns={
            'max_feasible_family_discount' : 'max_feasible_dg_discount',
            'min_feasible_family_discount' : 'min_feasible_dg_discount',
            'family_discount' : 'dg_discount'
        }
    )

elas = \
    pd.merge(\
        elas, 
        dg_elas, 
        how='inner',
        on = ['Location', 'OS', 'dg']\
    )

In [ ]:

elas['dg_satisfiable'] = \
    elas['min_feasible_dg_discount'] < elas['max_feasible_dg_discount']

desired_discount_below_bounds = \
    ( elas['min_feasible_family_discount'] > elas['dg_discount'] )

desired_discount_above_bounds = \
    ( elas['max_feasible_family_discount'] < elas['dg_discount'] )

desired_discount_oob =  \
    ( ( desired_discount_above_bounds ) | ( desired_discount_below_bounds ) )

elas['fixed_discount'] = \
    desired_discount_oob*0.0 + ~desired_discount_oob*elas['dg_discount']

elas['fixed_discount'] = \
    desired_discount_below_bounds*elas['min_feasible_family_discount'] + \
    desired_discount_above_bounds*elas['max_feasible_family_discount']

elas['min_price'] = \
    elas['dg_satisfiable']*elas['min_price'] + \
    ~elas['dg_satisfiable']*(( ( 1.0 - elas['fixed_discount'] ) * elas['od_core_price'] ))

elas['max_price'] = \
    elas['dg_satisfiable']*elas['max_price'] + \
    ~elas['dg_satisfiable']*( ( 1.0 - elas['fixed_discount'] ) * elas['od_core_price'] )


In [273]:
compete_fields = ['pre_process_family', 'OS']

_, compete_group_ids = np.unique(
    elas[compete_fields].apply(lambda x : '|'.join(x), axis=1).to_numpy(),
    return_inverse=True
)

elas['group'] = compete_group_ids

In [274]:
elas['group']

0         4
1         5
2         2
3        32
4        36
       ... 
7986    198
7987    199
7988    209
7989    200
7990    210
Name: group, Length: 7991, dtype: int64

In [275]:
elas['alpha_original'] = elas['alpha'].copy()
elas['beta_original'] = elas['beta'].copy()
elas['k_original'] = elas['k'].copy()
elas['weight_original'] = elas['weight'].copy()

elas['alpha'] = elas['opt']*elas["alpha"] + ~elas['opt']*0.0
elas['beta'] = elas['opt']*elas["beta"] + ~elas['opt']*0.0
elas['weight'] = elas['opt']*elas["weight"] + ~elas['opt']*0.0
elas['k'] = elas['opt']*elas["k"] + ~elas['opt']*0.1

In [276]:
def segment_sum(arr, segments):
    m = {}
    for i in range(len(segments)):
        if segments[i] not in m:
            m[segments[i]] = 0.0
            
        m[segments[i]] += arr[i]

    return np.array([m[s] for s in segments])

def demand_function(
    p: np.ndarray,
    k: np.ndarray,
    alpha: np.ndarray,
    beta: np.ndarray,
    weight: np.ndarray,
    group: np.ndarray,
    n_groups: int,
) -> np.ndarray:
    """
    @param p: price (vector)
    @param k: demand scaler (vector)
    @param alpha: alpha parameter (float or vector)
    @param beta: beta parameter (float or vector)
    @param weight: "weight" (float or vector)
    """
    return \
        (np.exp(segment_sum(np.log(p) * (alpha * weight), group))
        * k
        * np.power(p, beta - alpha * weight)
    )

In [277]:
results = []
print("Computing Min/Max Demand")

for group in list(elas['group'].unique()):
    restr = elas[elas['group'] == group].copy().reset_index(drop=True)

    tasks = list(
        restr[['pre_process_family', 'OS', 'Location']].drop_duplicates().apply(
            lambda x : {
                'pre_process_family' : x['pre_process_family'],
                'OS' : x['OS'],
                'Location' : x['Location']
            },
            axis=1
        )
    )

    for task in tasks:
        sub_restr = restr.copy()
        mask =  ((sub_restr['pre_process_family'] == task['pre_process_family']) & (sub_restr['Location'] == task['Location']) & (sub_restr['OS'] == task['OS']))

        sub_restr['p_min_demand'] = \
            mask*sub_restr['max_price'] + ~mask*sub_restr['min_price']

        sub_restr['p_max_demand'] = \
            mask*sub_restr['min_price'] + ~mask*sub_restr['max_price']

        sub_restr['group'] = 0

        sub_restr['min_demand'] = demand_function(
            p=sub_restr['p_min_demand'].to_numpy(),
            k=sub_restr['k'].to_numpy(),
            alpha=sub_restr['alpha'].to_numpy(),
            beta=sub_restr['beta'].to_numpy(),
            weight=sub_restr['weight'].to_numpy(),
            group=sub_restr['group'].to_numpy(),
            n_groups=1
        )

        sub_restr['max_demand'] = demand_function(
            p=sub_restr['p_max_demand'].to_numpy(),
            k=sub_restr['k'].to_numpy(),
            alpha=sub_restr['alpha'].to_numpy(),
            beta=sub_restr['beta'].to_numpy(),
            weight=sub_restr['weight'].to_numpy(),
            group=sub_restr['group'].to_numpy(),
            n_groups=1
        )

        results.append(
            {
                'pre_process_family' : task['pre_process_family'],
                'Location' : task['Location'],
                'OS' : task['OS'],
                'min_demand' : sub_restr[mask]['min_demand'].iloc[0],
                'max_demand' : sub_restr[mask]['max_demand'].iloc[0] 
            }
        )
        
print("Computed Min/Max Demand")

append_df = pd.DataFrame(data=results)
elas = \
    pd.merge(\
        elas,
        append_df,
        how='inner',
        on=['pre_process_family', 'Location', 'OS']
    )

elas['model_view_demand_pre_override'] = demand_function(
    p=elas['spot_core_price_pre_override'].to_numpy(),
    k=elas['k'].to_numpy(),
    alpha=elas['alpha'].to_numpy(),
    beta=elas['beta'].to_numpy(),
    weight=elas['weight'].to_numpy(),
    group=elas['group'].to_numpy(),
    n_groups=elas['group'].max() + 1
)

elas['model_view_demand_post_override'] = demand_function(
    p=elas['spot_core_price'].to_numpy(),
    k=elas['k'].to_numpy(),
    alpha=elas['alpha'].to_numpy(),
    beta=elas['beta'].to_numpy(),
    weight=elas['weight'].to_numpy(),
    group=elas['group'].to_numpy(),
    n_groups=elas['group'].max() + 1
)


Computing Min/Max Demand
Computed Min/Max Demand


In [278]:
demand = demand_function(
    p=elas['spot_core_price_pre_override'].to_numpy(),
    k=elas['k_original'].to_numpy(),
    alpha=elas['alpha_original'].to_numpy(),
    beta=elas['beta_original'].to_numpy(),
    weight=elas['weight_original'].to_numpy(),
    group=elas['group'].to_numpy(),
    n_groups=elas['group'].max() + 1
)

In [279]:
a = list(demand)

In [280]:
b = elas['avg_daily_demand_removed_nulls'].to_list()
d = \
    elas["k_original"]*elas["spot_core_price_pre_override"].pow(elas['beta_original'])*elas["prod_factors"]/elas["factor"]
d = d.to_list()

In [281]:
assert np.allclose(demand, elas['avg_daily_demand_removed_nulls'])

In [282]:
elas

,Location,pre_process_family,OS,weight_check_pre_override,spot_core_price_pre_override,od_core_price_pre_override,current_avg_discount_pre_override,min_spot_core_price_pre_override,max_spot_core_price_pre_override,min_od_core_price_pre_override,...,fixed_discount,group,alpha_original,beta_original,k_original,weight_original,min_demand,max_demand,model_view_demand_pre_override,model_view_demand_post_override
0,ae central,a,linux,1.0,0.061274,0.133350,0.540500,0.015439,0.096495,0.033600,...,0.49454,4,1.06,-2.19,0.0,0.0,0.0,0.0,0.0,0.0
1,ae central,a,windows,1.0,0.068837,0.101400,0.321148,0.014391,0.101835,0.021200,...,0.30000,5,0.68,-1.97,0.0,0.0,0.0,0.0,0.0,0.0
2,ae central,av2,linux,1.0,0.006783,0.067829,0.900000,0.005330,0.008525,0.053300,...,0.90000,2,1.06,-2.19,0.0,0.0,0.1,0.1,0.1,0.1
3,ae central,ddsv4,linux,1.0,0.009221,0.091821,0.899572,0.009214,0.009239,0.091750,...,0.90000,32,1.06,-2.19,0.0,0.0,0.1,0.1,0.1,0.1
4,ae central,ddv4,linux,1.0,0.009221,0.091821,0.899572,0.009214,0.009239,0.091750,...,0.90000,36,1.06,-2.19,0.0,0.0,0.1,0.1,0.1,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7986,za west,nda100v4,windows,1.0,0.150333,0.601330,0.750000,0.150333,0.150333,0.601330,...,0.00000,198,0.68,-1.97,0.0,0.0,0.0,0.0,0.0,0.0
7987,za west,ndma100v4,linux,1.0,0.143826,0.601608,0.760150,0.138833,0.148819,0.555330,...,0.00000,199,1.06,-2.19,0.0,0.0,0.0,0.0,0.0,0.0
7988,za west,ndv4a100s80nordma,linux,1.0,0.148819,0.647885,0.770300,0.148819,0.148819,0.647885,...,0.80810,209,1.06,-2.19,0.0,0.0,0.0,0.0,0.0,0.0
7989,za west,ndma100v4,windows,1.0,0.155518,0.647608,0.759200,0.150333,0.160704,0.601330,...,0.76840,200,0.68,-1.97,0.0,0.0,0.0,0.0,0.0,0.0


In [286]:
elas['key'] = elas[['Location', 'pre_process_family', 'OS']].apply(lambda x: '|'.join(x.values.astype(str)), axis=1)
sorted_keys = sorted(set(elas['key'].to_list()))

nkeys = len(sorted_keys)

In [288]:
key_inv_index = {}
for i in range(len(sorted_keys)):
    key_inv_index[sorted_keys[i]] = i

In [289]:
min_price = [float("Inf")]*len(sorted_keys)
max_price = [-float("Inf")]*len(sorted_keys)

keys = elas['key'].to_list()
minp = elas['min_price'].to_list()
maxp = elas['max_price'].to_list()

for i in range(len(keys)):
    j = key_inv_index[keys[i]]
    min_price[j] = minp[i]
    max_price[j] = maxp[i]

In [ ]:
dgs = elas['dg'].to_list()
od_price = elas['od_core_price'].to_list()

dg_leader = {}
dg_leader_od = {}

for i in range(len(keys)):
    dg = dgs[i]
    j = key_inv_index[keys[i]]
    
    if dg not in dg_leader:
        dg_leader[dg] = j
        dg_leader_od[dg] = od_price[i]

num_dgs = len(dg_leader)
price_eq = np.zeros((nkeys, nkeys))

for i in range(len(keys)):
    dg = dgs[i]
    
    j = key_inv_index[keys[i]]
    k = dg_leader[dg]
    q = dg_leader_od[dg]
    
    if k == j:
        price_eq[j, j] = 1.0
    else:
        price_eq[j, k] = od_price[i]/q

In [292]:
ineq_cons = {}

for i in range(len(price_constraints)):
    pcons = price_constraints[i]
    ineq_cons[(pcons['family_1'], pcons['family_2'])] = (pcons['a'], pcons['b'])

families = elas['pre_process_family'].to_list()

price_ineq_a = np.zeros((num_dgs, num_dgs))
price_ineq_b = np.zeros((num_dgs, num_dgs))

key_family_map = {}

for i in range(len(keys)):
    f = families[i]
    j = key_inv_index[keys[i]]
    key_family_map[j] = f

for i in range(len(keys)):
    dg1 = dgs[i]
    k = key_inv_index[keys[i]]
    f1 = key_family_map[k]
    u = dg_leader[dg1]
    
    for j in range(len(keys)):
        dg2 = dgs[j]
        l = key_inv_index[keys[j]]
        f2 = key_family_map[l]
        v = dg_leader[dg2]

        if (f1, f2) in ineq_cons:
            a, b = ineq_cons[(f1, f2)]

            
            
            price_ineq_a[k, l] = a
            price_ineq_b[k, l] = b
    

[{'family_1': 'dv2-1',
  'family_2': 'dv3',
  'a': 2.0,
  'b': 0.0,
  'desc': 'dv3 must be more than half the price of dv2-1'},
 {'family_1': 'dsv2-1',
  'family_2': 'dsv3',
  'a': 2.0,
  'b': 0.0,
  'desc': 'dsv3 must be more than half the price of dsv2-1'},
 {'family_1': 'dv3',
  'family_2': 'ddv4',
  'a': 1.0,
  'b': 0.0,
  'desc': 'dv3 must be less than the price of ddv4'},
 {'family_1': 'ddv4',
  'family_2': 'ddv5',
  'a': 1.0,
  'b': 0.0,
  'desc': 'ddv4 must be less than the price of ddv5'},
 {'family_1': 'ddsv4',
  'family_2': 'ddsv5',
  'a': 1.0,
  'b': 0.0,
  'desc': 'ddsv4 must be less than the price of ddsv5'},
 {'family_1': 'dv4',
  'family_2': 'dv5',
  'a': 1.0,
  'b': 0.0,
  'desc': 'dv4 must be less than the price of dv5'},
 {'family_1': 'dsv4',
  'family_2': 'dsv5',
  'a': 1.0,
  'b': 0.0,
  'desc': 'dsv4 must be less than the price of dsv5'},
 {'family_1': 'edv4',
  'family_2': 'edv5',
  'a': 1.0,
  'b': 0.0,
  'desc': 'edv4 must be less than the price of edv5'},
 {'f

In [285]:
x.groupby(by=["a"], as_index=False).agg(c=("b", "sum"))

AttributeError: 'str' object has no attribute 'groupby'

In [ ]:
x["b"] = 2*x["b"]

In [ ]:
len(elas)